In [ ]:
import wandb
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import torch
import kagglehub
import os
import torch.functional as F
import torch.nn as nn
import seaborn as sns
from torch import optim
from torchvision import datasets, transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader, Dataset, random_split
from sklearn.metrics import confusion_matrix, classification_report, precision_score, recall_score, f1_score
from PIL import Image
import warnings
warnings.filterwarnings("ignore")

In [ ]:
sweep_config = {
    'method': 'grid',
    'metric': {
        'name': 'validation_accuracy',
        'goal': 'maximize'
    },
    'parameters': {
        'learning_rate': {'values': [0.001, 0.01]},
        'batch_size': {'values': [16, 64]},
        'hidden_nodes': {'values': [128]},
        'image_size': {'values': [16, 224]},
        'num_epochs': {'values': [10, 20]}
    }
}

sweep_id = wandb.sweep(sweep_config, project='Computer_Vision_Simple_NN_Activation')

Create sweep with ID: fij0tuu6
Sweep URL: https://wandb.ai/pro-laxmi-indian-institute-of-technology-gandhinagar/Computer_Vision_Simple_NN_Activation/sweeps/fij0tuu6


In [ ]:
def train():
    with wandb.init() as run:
        config = wandb.config

        # Hyperparameters
        learning_rate = config.learning_rate
        batch_size = config.batch_size
        hidden_nodes = config.hidden_nodes
        image_size = config.image_size
        num_epochs = config.num_epochs

        # Data transformations
        mean = [0.485, 0.456, 0.406]
        std = [0.229, 0.224, 0.225]
        transform = transforms.Compose([
            transforms.Resize((image_size, image_size)),
            transforms.ToTensor(),
            transforms.Normalize(mean, std)
        ])

        # Load datasets
        path = kagglehub.dataset_download("alxmamaev/flowers-recognition")
        dataset_path = os.path.join(path, "flowers")
        dataset = ImageFolder(root=dataset_path, transform=transform)
        train_data, test_data, val_data = random_split(dataset, [0.8, 0.1, 0.1])
        train_dataset = DataLoader(train_data, batch_size=batch_size, shuffle=True)
        test_dataset = DataLoader(test_data, shuffle=True)
        val_dataset = DataLoader(val_data, shuffle=True)

        # Define model, loss function, optimizer here
        class model(nn.Module):
        # Input layer (224*3) inputs ---> Hidden layer 1 (128) --> Output Layer (5) --> Softmax(Output)
            def __init__(self, in_feature = image_size*image_size*3, hidden_layer_1 = hidden_nodes, out_features = 5):
                super().__init__()    # Instantiate our nn.module class
                self.fc1 = nn.Linear(in_feature, hidden_layer_1)
                self.out = nn.Linear(hidden_layer_1, out_features)
                self.relu = nn.ReLU()
            def forward(self, x):
                x = self.relu(self.fc1(x))
                x = self.out(x)
                return x

        torch.manual_seed(41)
        Model = model()
        criterion = nn.CrossEntropyLoss()
        optimizer = optim.Adam(Model.parameters(), lr = learning_rate)
        loss_values=[]
        training_accuracy = []
        validation_accuracy = []
        Model.train()

        for epoch in range(num_epochs):
            running_loss = 0.0  # Reset for each epoch

            for images, labels in train_dataset:
                # 1. Forward pass
                outputs = Model(images.permute(0,2,3,1).reshape(images.size(0), -1))
                loss = criterion(outputs, labels)

                # 2. Backward pass and optimization
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()

                # 3. ACCUMULATE LOSS (Crucial Step)
                running_loss += loss.item()

            # Calculate average loss for the epoch
            avg_loss = running_loss / len(train_dataset)
            loss_values.append(avg_loss)

            # Evaluation of the model on every epoch on validation data
            pred_actual_pairs_val = []
            with torch.no_grad():
                for images, label in val_dataset:
                    y_eval = Model(images.permute(0,2,3,1).reshape(images.size(0), -1))
                    for i in range(len(y_eval)):
                        pred_actual_pairs_val.append((torch.argmax(y_eval[i]).item(), int(label[i])))
            y_pred = np.array([p for p, a in pred_actual_pairs_val])
            y_true = np.array([a for p, a in pred_actual_pairs_val])
            validation_accuracy_instance = (y_pred == y_true).mean()
            validation_accuracy.append(validation_accuracy_instance)

            # Evaluation of the model on every epoch on training data
            pred_actual_pairs_train = []
            with torch.no_grad():
                for images, label in train_dataset:
                    y_eval = Model(images.permute(0,2,3,1).reshape(images.size(0), -1))
                    for i in range(len(y_eval)):
                        pred_actual_pairs_train.append((torch.argmax(y_eval[i]).item(), int(label[i])))
            y_pred = np.array([p for p, a in pred_actual_pairs_train])
            y_true = np.array([a for p, a in pred_actual_pairs_train])
            training_accuracy_instance = (y_pred == y_true).mean()
            training_accuracy.append(training_accuracy_instance)

            print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {avg_loss:.4f}, Training Accuracy: {training_accuracy_instance:.4f}, Validation Accuracy: {validation_accuracy_instance:.4f}")
            wandb.log({
                'epoch': epoch+1,
                'loss': avg_loss,
                'training_accuracy': training_accuracy_instance,
                'validation_accuracy': validation_accuracy_instance
            })
        # Final evaluation on test data
        pred_actual_pairs_test = []
        with torch.no_grad():
            for images, label in test_dataset:
                y_eval = Model(images.permute(0,2,3,1).reshape(images.size(0), -1))
                for i in range(len(y_eval)):
                    pred_actual_pairs_test.append((torch.argmax(y_eval[i]).item(), int(label[i])))
        y_pred = np.array([p for p, a in pred_actual_pairs_test])
        y_true = np.array([a for p, a in pred_actual_pairs_test])
        test_accuracy = (y_pred == y_true).mean()
        print(f"Test Accuracy: {test_accuracy:.4f}")
        wandb.log({'test_accuracy': test_accuracy})
wandb.agent(sweep_id, train)

wandb: Agent Starting Run: pxwqrpii with config:
wandb: 	batch_size: 16
wandb: 	hidden_nodes: 128
wandb: 	image_size: 16
wandb: 	learning_rate: 0.001
wandb: 	num_epochs: 10
wandb: Currently logged in as: pro-laxmi (pro-laxmi-indian-institute-of-technology-gandhinagar) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Using Colab cache for faster access to the 'flowers-recognition' dataset.
Epoch [1/10], Loss: 1.3134, Training Accuracy: 0.5585, Validation Accuracy: 0.4640
Epoch [2/10], Loss: 1.1690, Training Accuracy: 0.5683, Validation Accuracy: 0.4640
Epoch [3/10], Loss: 1.0832, Training Accuracy: 0.6077, Validation Accuracy: 0.4687
Epoch [4/10], Loss: 1.0169, Training Accuracy: 0.6236, Validation Accuracy: 0.4965
Epoch [5/10], Loss: 0.9615, Training Accuracy: 0.6697, Validation Accuracy: 0.4756
Epoch [6/10], Loss: 0.9055, Training Accuracy: 0.7180, Validation Accuracy: 0.5104
Epoch [7/10], Loss: 0.8550, Training Accuracy: 0.7458, Validation Accuracy: 0.4942
Epoch [8/10], Loss: 0.7857, Training Accuracy: 0.7334, Validation Accuracy: 0.5012
Epoch [9/10], Loss: 0.7688, Training Accuracy: 0.7690, Validation Accuracy: 0.4872
Epoch [10/10], Loss: 0.6889, Training Accuracy: 0.7930, Validation Accuracy: 0.5220
Test Accuracy: 0.5278


epoch,▁▂▃▃▄▅▆▆▇█
loss,█▆▅▅▄▃▃▂▂▁
test_accuracy,▁
training_accuracy,▁▁▂▃▄▆▇▆▇█
validation_accuracy,▁▁▂▅▂▇▅▅▄█
epoch,10
loss,0.68886
test_accuracy,0.52778
training_accuracy,0.79299
validation_accuracy,0.52204


wandb: Agent Starting Run: 8k9shslx with config:
wandb: 	batch_size: 16
wandb: 	hidden_nodes: 128
wandb: 	image_size: 16
wandb: 	learning_rate: 0.001
wandb: 	num_epochs: 20


Using Colab cache for faster access to the 'flowers-recognition' dataset.
Epoch [1/20], Loss: 1.3141, Training Accuracy: 0.5000, Validation Accuracy: 0.4246
Epoch [2/20], Loss: 1.1492, Training Accuracy: 0.5819, Validation Accuracy: 0.4733
Epoch [3/20], Loss: 1.0879, Training Accuracy: 0.5953, Validation Accuracy: 0.4640
Epoch [4/20], Loss: 1.0154, Training Accuracy: 0.6410, Validation Accuracy: 0.4849
Epoch [5/20], Loss: 0.9528, Training Accuracy: 0.6659, Validation Accuracy: 0.4664
Epoch [6/20], Loss: 0.8786, Training Accuracy: 0.6772, Validation Accuracy: 0.4803
Epoch [7/20], Loss: 0.8466, Training Accuracy: 0.7412, Validation Accuracy: 0.4780
Epoch [8/20], Loss: 0.7939, Training Accuracy: 0.7499, Validation Accuracy: 0.5035
Epoch [9/20], Loss: 0.7276, Training Accuracy: 0.7730, Validation Accuracy: 0.4919
Epoch [10/20], Loss: 0.6983, Training Accuracy: 0.7962, Validation Accuracy: 0.4780
Epoch [11/20], Loss: 0.6280, Training Accuracy: 0.7985, Validation Accuracy: 0.4803
Epoch [12/2

...

wandb: Sweep Agent: Waiting for job.
wandb: Job received.
wandb: Agent Starting Run: i1yc3uyi with config:
wandb: 	batch_size: 16
wandb: 	hidden_nodes: 128
wandb: 	image_size: 224
wandb: 	learning_rate: 0.01
wandb: 	num_epochs: 10


Using Colab cache for faster access to the 'flowers-recognition' dataset.
Epoch [1/10], Loss: 200.5023, Training Accuracy: 0.3411, Validation Accuracy: 0.3434
Epoch [2/10], Loss: 229.0192, Training Accuracy: 0.3115, Validation Accuracy: 0.2761
Epoch [3/10], Loss: 259.9560, Training Accuracy: 0.3263, Validation Accuracy: 0.2738
Epoch [4/10], Loss: 274.1236, Training Accuracy: 0.3057, Validation Accuracy: 0.2831
Epoch [5/10], Loss: 381.3851, Training Accuracy: 0.3358, Validation Accuracy: 0.2877
Epoch [6/10], Loss: 313.8312, Training Accuracy: 0.3888, Validation Accuracy: 0.3109
Epoch [7/10], Loss: 352.9921, Training Accuracy: 0.3660, Validation Accuracy: 0.2599
Epoch [8/10], Loss: 438.8680, Training Accuracy: 0.4094, Validation Accuracy: 0.3086
Epoch [9/10], Loss: 294.7233, Training Accuracy: 0.3874, Validation Accuracy: 0.2599
Epoch [10/10], Loss: 304.3506, Training Accuracy: 0.3822, Validation Accuracy: 0.3109
Test Accuracy: 0.3009


epoch,▁▂▃▃▄▅▆▆▇█
loss,▁▂▃▃▆▄▅█▄▄
test_accuracy,▁
training_accuracy,▃▁▂▁▃▇▅█▇▆
validation_accuracy,█▂▂▃▃▅▁▅▁▅
epoch,10
loss,304.35062
test_accuracy,0.30093
training_accuracy,0.38217
validation_accuracy,0.3109


wandb: Agent Starting Run: dyfp5ar6 with config:
wandb: 	batch_size: 16
wandb: 	hidden_nodes: 128
wandb: 	image_size: 224
wandb: 	learning_rate: 0.01
wandb: 	num_epochs: 20


Using Colab cache for faster access to the 'flowers-recognition' dataset.
Epoch [1/20], Loss: 167.8490, Training Accuracy: 0.2710, Validation Accuracy: 0.2575
Epoch [2/20], Loss: 299.3780, Training Accuracy: 0.3358, Validation Accuracy: 0.3039
Epoch [3/20], Loss: 307.1302, Training Accuracy: 0.2988, Validation Accuracy: 0.2831
Epoch [4/20], Loss: 333.5492, Training Accuracy: 0.3283, Validation Accuracy: 0.2923
Epoch [5/20], Loss: 285.7363, Training Accuracy: 0.3532, Validation Accuracy: 0.2947
Epoch [6/20], Loss: 409.4133, Training Accuracy: 0.3463, Validation Accuracy: 0.2807
Epoch [7/20], Loss: 297.1611, Training Accuracy: 0.3570, Validation Accuracy: 0.2807


In [1]:
print("Sweep Complete.")
print(""""Best test accuracy achieved during the sweep can be found in the Weights & Biases dashboard, which is " \
    Image size: 16
    Batch size: 16
    Hidden nodes: 128
    Learning rate: 0.001
    Num epochs: 10""")

Sweep Complete.
"Best test accuracy achieved during the sweep can be found in the Weights & Biases dashboard, which is "     Image size: 16
    Batch size: 16
    Hidden nodes: 128
    Learning rate: 0.001
    Num epochs: 10
